# InstaGeo Demo

<a href="https://colab.research.google.com/github/instadeepai/InstaGeo-E2E-Geospatial-ML/blob/main/notebooks/InstaGeo_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from getpass import getpass
import os

In [ ]:
repository_url = "https://github.com/instadeepai/InstaGeo-E2E-Geospatial-ML"

!git clone {repository_url}

In [ ]:
%%bash
cd InstaGeo-E2E-Geospatial-ML
pip install -e .[all]

## EarthData Login

In [ ]:
# Create an EarthData user account on https://urs.earthdata.nasa.gov/
USERNAME = getpass('Enter your EarthDAata username: ')
PASSWORD = getpass('Enter your EarthDAata password: ')

content = f"""machine urs.earthdata.nasa.gov login {USERNAME} password {PASSWORD}"""

with open(os.path.expanduser('~/.netrc'), 'w') as file:
    file.write(content)

## InstaGeo - Data

- Searches and retrieves HLS metadata. 
- Downloads the required bands from HLS granules
- Creates Chips and Target

In [ ]:
%%bash
mkdir demo
python -m "instageo.data.chip_creator" \
    --dataframe_path="InstaGeo-E2E-Geospatial-ML/tests/data/test_breeding_data.csv" \
    --output_directory="demo" \
    --min_count=4 \
    --chip_size=224 \
    --no_data_value=-1 \
    --temporal_tolerance=3 \
    --temporal_step=30 \
    --mask_cloud=False \
    --num_steps=3

## InstaGeo - Model

Create a dataframe for created chips and target that will be used with model component

In [ ]:
import os
import os
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
root_dir = Path.cwd()
chips_orig = os.listdir(os.path.join(root_dir, "demo/chips"))
chips = [chip.replace("chip", "chips/chip") for chip in chips_orig]
seg_maps = [chip.replace("chip", "seg_maps/seg_map") for chip in chips_orig]

df = pd.DataFrame({"Input": chips, "Label": seg_maps})
df.to_csv(os.path.join(root_dir, "demo/demo_train.csv"))
df.to_csv(os.path.join(root_dir, "demo/demo_val.csv"))

Launch training

In [ ]:
# %%bash
!python -m instageo.model.run --config-name=locust \
    root_dir='demo' \
    train.batch_size=8 \
    train.num_epochs=3 \
    train_filepath="demo/demo_train.csv" \
    valid_filepath="demo/demo_val.csv"

Run Model Evaluation

In [ ]:
# %%bash
!python -m instageo.model.run --config-name=locust \
    root_dir='demo' \
    test_filepath="demo/demo_val.csv" \
    train.batch_size=8 \
    checkpoint_path='add-checkpoint-path' \
    mode=eval

**Run Inference**

- Runs inference on the Africa continent (plus some part of Asia)
- Downloads and run inference on 2,120 HLS tiles

In [ ]:
!gsutil -m cp -r gs://instageo/checkpoints/locust_breeding_ckpt .
!gsutil cp gs://instageo/utils/africa_prediction_template.csv .
!mkdir -p inference/2023-01

**Create Inference Data**

During inference we only download HLS tiles and irectly runinference on them using the sliding window inference feature.

In [ ]:
!python -m "instageo.data.chip_creator" \
    --dataframe_path="africa_prediction_template.csv" \
    --output_directory="inference/2023-01" \
    --min_count=1 \
    --no_data_value=-1 \
    --temporal_tolerance=3 \
    --temporal_step=30 \
    --num_steps=3 \
    --download_only

**Run Inference**

In [ ]:
!python -m instageo.model.run --config-name=locust \
    root_dir='inference/2023-01' \
    test_filepath='hls_dataset.json' \
    train.batch_size=16 \
    test.mask_cloud=True \
    checkpoint_path='add-checkpoint-path' \
    mode=predict

## InstaGeo - Apps

Visualize predictions my moving HLS prediction GeoTiff files to an appropriate directory.

In [ ]:
!mkdir -p predictions/2023/1
!mv inference/2023-01/predictions/* content/predictions/2023/1

In [ ]:
!npm install localtunnel

In [ ]:
!nohup streamlit run InstaGeo/instageo/apps/app.py --server.address=localhost &

Retrieve your IP address which is the password of the localtunnel

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
!npx localtunnel --port 8501